In [1]:
import cameo as cameo
from cobra import Model, Reaction, Metabolite
from collections import OrderedDict
import re
import escher
from cameo import load_model
from cameo import Reaction, Metabolite
from cameo.data import metanetx
from cameo.network_analysis import model_to_network, remove_highly_connected_nodes
from cameo.flux_analysis.analysis import phenotypic_phase_plane
from cameo.parallel import SequentialView
import os

In [2]:
relative_directory = os.getcwd()
filename = relative_directory + '/iCL1079.xml'
model = cameo.load_model(filename,sanitize=False)

/Users/clie/Dev/cobrapy/cobra/io/sbml.py:205: UserWarning:

PROTON_c appears as a reactant and product RXN__45__10981

/Users/clie/Dev/cobrapy/cobra/io/sbml.py:205: UserWarning:

ASCORBATE_c appears as a reactant and product RXN__45__12440

/Users/clie/Dev/cobrapy/cobra/io/sbml.py:205: UserWarning:

PROTON_c appears as a reactant and product RXN__45__6081

/Users/clie/Dev/cobrapy/cobra/io/sbml.py:205: UserWarning:

PROTON_m appears as a reactant and product NADH__45__DEHYDROG__45__A__45__RXN

/Users/clie/Dev/cobrapy/cobra/io/sbml.py:205: UserWarning:

PROTON_m appears as a reactant and product TRANS__45__RXN__45__249

/Users/clie/Dev/cobrapy/cobra/io/sbml.py:205: UserWarning:

PROTON_c appears as a reactant and product _3__46__6__46__3__46__10__45__RXN

/Users/clie/Dev/cobrapy/cobra/io/sbml.py:205: UserWarning:

PROTON_c appears as a reactant and product RXN0__45__3281

/Users/clie/Dev/cobrapy/cobra/io/sbml.py:205: UserWarning:

WATER_c appears as a reactant and product _1__46__14__46_

### Check for Unbalanced Reactions

In [3]:
import imp

In [4]:
foo = imp.load_source('curation_tools','/Users/clie/Desktop/EFPro2/Functions_Modules/curation_tools.py')

In [5]:
Unbalanced_reactions = foo.find_unbalanced_reactions(model)

There are 0 unbalanced reactions and 0 metabolites with a faulty syntax


In [6]:
print Unbalanced_reactions[0].keys()
for rxnid in Unbalanced_reactions[0].keys():
    model.reactions.get_by_id(rxnid).upper_bound = 0
    model.reactions.get_by_id(rxnid).lower_bound = 0

[]


In [7]:
mets_noformula = [met for met in model.metabolites if met.formula == '']

In [8]:
len(mets_noformula)

441

### Map the Met/Rxn IDs from Metacyc to MNX to BiGG

#### Import of the mapping table for MetaCyc to MNX

In [9]:
import pandas as pd

In [10]:
# Metacyc as keys
chem_xref_MNX_Metacyc = pd.read_csv('/Users/clie/Desktop/EFPro2/Calculation Results/MetanetX Mapping/chem_xref_2_alt.tsv.txt', delimiter='\t')
df_Metacyc = chem_xref_MNX_Metacyc.loc[192554:208025,'XRef':'MNX_ID']

In [11]:
reindex = df_Metacyc.set_index('XRef_ID')

In [12]:
df = reindex.drop('XRef', 1)

In [13]:
chem_xref_MNX_Metacyc_dict = df.to_dict('index')

In [14]:
MetCyc2MNX= dict([(k,chem_xref_MNX_Metacyc_dict[k]['MNX_ID']) for k in chem_xref_MNX_Metacyc_dict.keys()])

In [15]:
# Get a mapping from Metacyc to the new MNXR IDs.
reac_xref_MNX_2 = pd.read_csv('/Users/clie/Desktop/EFPro2/Calculation Results/MetanetX Mapping/reac_xref_2_alt.tsv.txt', delimiter='\t')
df_2 = reac_xref_MNX_2.loc[22929:36711,'XRef':'MNX_ID']
reindex_rxn = df_2.set_index('XRef_ID')
df2 = reindex_rxn.drop('XRef', 1)
rxn_xref_MNX_Metacyc_dict = df2.to_dict('index')
RxnCyc2MNX= dict([(k,rxn_xref_MNX_Metacyc_dict[k]['MNX_ID']) for k in rxn_xref_MNX_Metacyc_dict.keys()])

In [16]:
RxnCyc2MNX

{'GLUCOSE-1-PHOSPHATE-PHOSPHODISMUTASE-RXN': 'MNXR32864',
 'RXN-12158': 'MNXR21956',
 'D-XYLULOSE-REDUCTASE-RXN': 'MNXR84426',
 'HALOALKANE-DEHALOGENASE-RXN': 'MNXR37148',
 'RXN-13525': 'MNXR87852',
 'PROCOLLAGEN-GALACTOSYLTRANSFERASE-RXN': 'MNXR87177',
 'RXN-14471': 'MNXR69704',
 'RXN-11109': 'MNXR87365',
 'RXN-11108': 'MNXR71565',
 'RXN-12157': 'MNXR87636',
 'RXN-11105': 'MNXR87364',
 'RXN-11104': 'MNXR87363',
 'RXN-11107': 'MNXR21230',
 'RXN-11106': 'MNXR8789',
 'RXN-11101': 'MNXR9011',
 '3.6.3.33-RXN': 'MNXR19984',
 'RXN-11103': 'MNXR34082',
 'RXN-11102': 'MNXR8981',
 '1.3.99.17-RXN': 'MNXR26398',
 'RXN-12718': 'MNXR76953',
 'PNKIN-RXN': 'MNXR4539',
 '3.4.21.49-RXN': 'MNXR86818',
 'RXN-4229': 'MNXR17302',
 'RXN-4228': 'MNXR89835',
 'RXN-4225': 'MNXR17300',
 'RXN-12710': 'MNXR22325',
 'RXN-12713': 'MNXR5902',
 'RXN-4226': 'MNXR89834',
 'RXN-12715': 'MNXR19078',
 'RXN-12714': 'MNXR85993',
 'GLU6PDEHYDROG-RXN': 'MNXR2456',
 'RXN-4222': 'MNXR77467',
 'RXN-15848': 'MNXR89058',
 'RXN-158

#### Encode the IDs in the Dictionary to match the ones in the model

In [17]:
MetCyc2MNX_fixed = {}
RxnCyc2MNX_fixed = {}

for key in MetCyc2MNX.keys():
    keyx = key.replace('(','__40__')
    keyx = keyx.replace(')','__41__')
    keyx = keyx.replace('-','__45__')
    keyx = keyx.replace('+','__43__')
    keyx = keyx.replace('.','__46__')
    keyx = keyx.replace('/','__47__')
    MetCyc2MNX_fixed.update({keyx: MetCyc2MNX[key]})
for key in RxnCyc2MNX.keys():
    keyx = key.replace('(','__40__')
    keyx = keyx.replace(')','__41__')
    keyx = keyx.replace('-','__45__')
    keyx = keyx.replace('+','__43__')
    keyx = keyx.replace('.','__46__')
    keyx = keyx.replace('/','__47__')
    RxnCyc2MNX_fixed.update({keyx: RxnCyc2MNX[key]})

#### Import of the mapping table from MNX to BiGG

In [18]:
# MNX as keys mapping to BiGG for Metabolites
chem_xref_MNX_BiGG = pd.read_csv('/Users/clie/Desktop/EFPro2/Calculation Results/MetanetX Mapping/chem_xref_2_alt.tsv.txt', delimiter='\t')
df_Metacyc = chem_xref_MNX_BiGG.loc[2:4042,'XRef':'MNX_ID']

In [19]:
reindex_mnxmet = df_Metacyc.set_index('MNX_ID')

In [20]:
df3 = reindex_mnxmet.drop('XRef', 1)

In [21]:
met_xref_MNX_BIGG_dict = df3.to_dict('index')
MetMNX2BIGG = dict([(k,met_xref_MNX_BIGG_dict[k]['XRef_ID']) for k in met_xref_MNX_BIGG_dict.keys()])

In [22]:
MetMNX2bigg = dict((v,k) for k,v in MetMNX2BIGG.iteritems())

In [23]:
# MNX as keys mapping to BiGG for reactions
chem_xref_MNX_BiGG = pd.read_csv('/Users/clie/Desktop/EFPro2/Calculation Results/MetanetX Mapping/reac_xref_2_alt.tsv.txt', delimiter='\t')
df_Metacyc1 = chem_xref_MNX_BiGG.loc[0:11459,'XRef':'MNX_ID']
reindex_mnxrxn = df_Metacyc1.set_index('MNX_ID')
df4 = reindex_mnxrxn.drop('XRef', 1)
rxn_xref_MNX_BIGG_dict = df4.to_dict('index')
RxnMNX2BIGG = dict([(k,rxn_xref_MNX_BIGG_dict[k]['XRef_ID']) for k in rxn_xref_MNX_BIGG_dict.keys()])

#### Construction of a mapping dict from MetaCyc straight to BiGG

In [24]:
# Metabolites
met_no_conversion_possible = []
MetCyc2bigg = {}

for key in MetCyc2MNX_fixed:
    if MetCyc2MNX_fixed[key] in MetMNX2BIGG.keys():
        mnx = MetCyc2MNX_fixed[key]
        MetCyc2bigg[key] = MetMNX2BIGG[mnx]
    else:
        met_no_conversion_possible.append(key)

In [25]:
# Reactions
rxn_no_conversion_possible = []
RxnCyc2bigg = {}

for key in RxnCyc2MNX_fixed:
    if RxnCyc2MNX_fixed[key] in RxnMNX2BIGG.keys():
        mnx = RxnCyc2MNX_fixed[key]
        RxnCyc2bigg[key] = RxnMNX2BIGG[mnx]
    else:
        rxn_no_conversion_possible.append(key)

#### Amount of MetaCyc IDs that cannot be converted to BiGG IDs via MNX IDs

In [26]:
print len(met_no_conversion_possible)
print len(rxn_no_conversion_possible)

13517
12001


#### Amount of MetaCyc IDs that could be mapped

In [27]:
print len(MetCyc2bigg.keys())
print len(RxnCyc2bigg.keys())

1955
1782


#### Find ambiguous mappings in the dict that are relevant for the IDs in the model

In [28]:
what_the = {x.id:MetCyc2bigg[x.id.lstrip('_')[:-2]] for x in list(model.metabolites) if x.id.lstrip('_')[:-2] in MetCyc2bigg.keys()}

In [29]:
ambig_dict = {}
for key in what_the.keys(): 
    if what_the.values().count(what_the[key]) > 1:
        if what_the[key] not in ambig_dict.keys():
            ambig_dict[what_the[key]] = [key[:-2]]
        else:
            if key[:-2] in ambig_dict[what_the[key]]:
                pass
            else:
                ambig_dict[what_the[key]].append(key[:-2])

In [30]:
print [x for x in ambig_dict.keys() if len(ambig_dict[x]) > 1]

['nadp', 'Lfmkynr', 'phpyr', 'acac', 'nadph', 'coa', 'so3', 'nh3', 'trnagln', 'op4en', 'oh1']


In [31]:
ambig_dict['so3']

['SO3', 'HSO3']

In [32]:
what_the['HSO3_c'] = 'hso3'

In [33]:
ambig_dict['nadp']

['NAD__45__P__45__OR__45__NOP', 'NADP']

In [34]:
what_the['NAD__45__P__45__OR__45__NOP_c'] = 'NAD__45__P__45__OR__45__NOP'

In [35]:
ambig_dict['phpyr']

['ENOL__45__PHENYLPYRUVATE', 'PHENYL__45__PYRUVATE']

In [36]:
what_the['ENOL__45__PHENYLPYRUVATE_c'] = 'ENOL__45__PHENYLPYRUVATE'

In [37]:
ambig_dict['acac']

['A__45__3__45__OXO__45__ACID', '_3__45__KETOBUTYRATE']

In [38]:
what_the['A__45__3__45__OXO__45__ACID_c'] = 'A__45__3__45__OXO__45__ACID'

In [39]:
ambig_dict['oh1']

['OH', 'WATER']

In [40]:
what_the['WATER_e'] = 'WATER'

In [41]:
what_the['WATER_m'] = 'WATER'

In [42]:
what_the['WATER_c'] = 'WATER'

In [43]:
ambig_dict['op4en']

['OXOPENTENOATE', 'CPD__45__6761']

In [44]:
what_the['CPD__45__6761_c'] = 'CPD__45__6761'

In [45]:
ambig_dict['trnagln']

['Alcohols', 'GLN__45__tRNAs']

In [46]:
what_the['Alcohols_c'] = 'Alcohols'

In [47]:
ambig_dict['nh3']

['AMMONIUM', 'AMMONIA']

In [48]:
what_the['AMMONIUM_c'] = 'nh4'

In [49]:
what_the['AMMONIUM_e'] = 'nh4'

In [50]:
ambig_dict['coa']

['CO__45__A', 'COA__45__GROUP']

In [51]:
what_the['COA__45__GROUP_c'] = 'COA__45__GROUP_c'

In [52]:
ambig_dict['nadph']

['NADH__45__P__45__OR__45__NOP', 'NADPH']

In [53]:
what_the['NADH__45__P__45__OR__45__NOP_c'] = 'NADH__45__P__45__OR__45__NOP'

In [54]:
ambig_dict['Lfmkynr']

['N__45__FORMYLKYNURENINE', 'CPD__45__10488']

In [55]:
what_the['CPD__45__10488_c'] = 'CPD__45__10488'

#### Mapping the IDs in the current loaded version of the model

In [56]:
len(model.metabolites)

1987

In [57]:
len(what_the)

1103

In [58]:
for met in what_the.keys():
    try:
        new_id = what_the[met] + '_' + model.metabolites.get_by_id(met).compartment
        model.metabolites.get_by_id(met).id = new_id
    except ValueError:
        print met, what_the[met]

CARBON__45__DIOXIDE_m co2


### Manually fixing those mets that couldn't be matched

In [59]:
def swap_mets(metA, metB):
	for reaction in metA.reactions:
		coef = reaction.metabolites[metA]
		reaction.add_metabolites({metA: -coef, metB: coef}, combine=True)

In [60]:
foo.find_reaction_from_metabolites(model,['CARBON__45__DIOXIDE_m', 'co2_m'])

--------------------------------------------------
These metabolites have no common reactions!


In [61]:
model.metabolites.co2_m.reactions

frozenset({<Reaction TRANS_Permeable_Small_Uncharged_Molecules_CO2_EX at 0x126f9c650>})

In [62]:
model.metabolites.CARBON__45__DIOXIDE_m.reactions

frozenset({<Reaction _2OXOGLUTARATEDEH__45__RXN at 0x11cd1e110>,
           <Reaction ISOCITDEH__45__RXN at 0x126808b50>,
           <Reaction ISOCITRATE__45__DEHYDROGENASE__45__NAD__43____45__RXN at 0x126be5050>,
           <Reaction PYRUVDEH__45__RXN at 0x126ce66d0>,
           <Reaction _1__46__1__46__1__46__39__45__RXN at 0x126d34b50>,
           <Reaction MALIC__45__NADP__45__RXN at 0x126d34c90>,
           <Reaction TRANS_Permeable_Small_Uncharged_Molecules_CO2 at 0x126f9c510>,
           <Reaction Carbonic_Anhydrase at 0x126fa8fd0>})

In [63]:
swap_mets(model.metabolites.CARBON__45__DIOXIDE_m,model.metabolites.co2_m)
model.metabolites.CARBON__45__DIOXIDE_m.remove_from_model('subtractive')

In [64]:
model.metabolites.WATER_m.id = 'h2o_m'

In [65]:
model.metabolites.WATER_e.id = 'h2o_e'

In [66]:
model.metabolites.WATER_c.id = 'h2o_c'

In [67]:
print len(model.metabolites)
print len(what_the)
print '%f percent could be mapped' % (float(len(what_the)) / float(len(model.metabolites)) * 100)

1986
1103
55.538771 percent could be mapped


### Mapping of reactions

In [68]:
what_the_rxn = {x.id:RxnCyc2bigg[x.id.lstrip('_')] for x in list(model.reactions) if x.id.lstrip('_') in RxnCyc2bigg.keys()}

In [69]:
for rxn in what_the_rxn.keys():
    try:
        model.reactions.get_by_id(rxn).id = what_the_rxn[rxn]
    except ValueError:
        print rxn, what_the_rxn[rxn]

TRANS__45__RXN__45__143 Ktex
RXN__45__13698 ALATA_Lm


In [70]:
model.remove_reactions([model.reactions.RXN__45__13698])

In [71]:
print len(model.reactions)
print len(what_the_rxn)
print '%f percent could be mapped' % (float(len(what_the_rxn)) / float(len(model.reactions)) * 100)

2217
595
26.838069 percent could be mapped


In [72]:
Rxns_with_Metacyc_ID = [x for x in model.reactions if not x.id in RxnCyc2bigg.values()]

#### Adding in the growth conditions from the FBA Script that came with the Thesis

In [73]:
for rxn in model.reactions:
    if rxn.reversibility == False:
        rxn.lower_bound = 0
        rxn.upper_bound = 25
    else:
        rxn.lower_bound = -25
        rxn.upper_bound = 25   

In [74]:
## 
## Uptake Vitamins
rxn = Reaction('BTNtex')
rxn.name = 'Biotin transport via diffusion (extracellular to cytosol)'
model.add_reaction(rxn)
rxn.build_reaction_from_string('btn_e <=> btn_c')

rxn = Reaction('PYDXtex')
rxn.name = 'Pyridoxal transport via diffusion (extracellular to cytosol)'
model.add_reaction(rxn)
rxn.build_reaction_from_string('pydx_e <=> pydx_c')

rxn = Reaction('THMtex')
rxn.name = 'Thiamin transport via diffusion (extracellular to cytosol)'
model.add_reaction(rxn)
rxn.build_reaction_from_string('thm_e <=> thm_c')

rxn = Reaction('PNTO_Rtex')
rxn.name = '(R)-Pantothenate transport via diffusion (extracellular to cytosol)'
model.add_reaction(rxn)
rxn.build_reaction_from_string('pnto_R_e <=> pnto_R_c')

model.reactions.TRANS__45__RXNTFS__45__17.lower_bound = -25
model.reactions.TRANS__45__RXNTFS__45__17.upper_bound = 25

model.metabolites.btn_e.formula = 'C10H15N2O3S'
model.metabolites.pydx_e.formula = 'C8H9NO3'
model.metabolites.thm_e.formula = 'C12H17N4OS'
model.metabolites.pnto_R_e.formula = 'C9H16NO5'

unknown metabolite 'btn_e' created
unknown metabolite 'pydx_e' created
unknown metabolite 'thm_e' created
unknown metabolite 'pnto_R_e' created
unknown metabolite 'pnto_R_c' created


In [75]:
model.reactions.EX_cl_e.id = 'CLtex'

In [76]:
## Medium Minerals
rxn = Reaction('EX_o2_e')
rxn.name = 'Oxygen exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('o2_e <=> ')

rxn = Reaction('CO2tex')
rxn.name = 'Carbon Dioxide transport via diffusion (extracellular to cytosol)'
model.add_reaction(rxn)
rxn.build_reaction_from_string('co2_e <=> co2_c')

rxn = Reaction('EX_co2_e')
rxn.name = 'Carbon Dioxide exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('co2_e <=>')

rxn = Reaction('EX_nh4_e')
rxn.name = 'Ammonium exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('nh4_e <=> ')

rxn = Reaction('EX_cl_e')
rxn.name = 'Chloride exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('cl_e <=> ')

rxn = Reaction('EX_k_e')
rxn.name = 'Potassium exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('k_e <=> ')

rxn = Reaction('EX_pi_e')
rxn.name = 'Phosphate exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('pi_e <=> ')

rxn = Reaction('EX_mg2_e')
rxn.name = 'Magnesium exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('mg2_e <=> ')

rxn = Reaction('EX_so4_e')
rxn.name = 'Sulfate exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('so4_e <=> ')

rxn = Reaction('EX_h2o_e')
rxn.name = 'Water exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('h2o_e <=> ')

unknown metabolite 'co2_e' created


In [77]:
## Medium Trace Elements
rxn = Reaction('EX_zn2_e')
rxn.name = 'Zinc exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('zn2_e <=> ')

rxn = Reaction('EX_fe2_e')
rxn.name = 'Iron exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('fe2_e <=> ')

rxn = Reaction('EX_cobalt2_e')
rxn.name = 'Cobalt exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('cobalt2_e <=> ')

rxn = Reaction('EX_copper2_e')
rxn.name = 'Copper exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('copper2_e <=> ')

rxn = Reaction('EX_na1_e')
rxn.name = 'Sodium exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('na1_e <=> ')

rxn = Reaction('EX_ca2_e')
rxn.name = 'Calcium exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('ca2_e <=> ')

unknown metabolite 'copper2_e' created


In [78]:
## Medium Vitamins
rxn = Reaction('EX_btn_e')
rxn.name = 'Biotin exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('btn_e <=> ')
rxn.lower_bound = -5
rxn.upper_bound = 1000

rxn = Reaction('EX_pnto_R_e')
rxn.name = '(R)-Pantothenate exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('pnto_R_e <=> ')
rxn.lower_bound = -5
rxn.upper_bound = 1000

rxn = Reaction('EX_nac_e')
rxn.name = 'Nicotinic acid exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('nac_e <=> ')
rxn.lower_bound = -5
rxn.upper_bound = 1000

rxn = Reaction('EX_inost_e')
rxn.name = '(Myo)-Inositol exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('inost_e <=> ')
rxn.lower_bound = -5
rxn.upper_bound = 1000

rxn = Reaction('EX_thm_e')
rxn.name = 'Thiamine exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('thm_e <=> ')
rxn.lower_bound = -5
rxn.upper_bound = 1000

rxn = Reaction('EX_pydx_e')
rxn.name = 'Pyridoxal exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('pydx_e <=> ')
rxn.lower_bound = -5
rxn.upper_bound = 1000

rxn = Reaction('EX_4abz_e')
rxn.name = '4-Aminobenzoate exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('4abz_e <=> ')
rxn.lower_bound = -5
rxn.upper_bound = 1000

In [79]:
## Medium Carbon Source 
rxn = Reaction('EX_glc_A_e')
rxn.name = 'D-Glucose exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('glc_A_e <=> ')
rxn.lower_bound = -20
rxn.upper_bound = 1000

rxn = Reaction('EX_glyc_e')
rxn.name = 'Glycerol exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('glyc_e <=> ')
rxn.lower_bound = 0
rxn.upper_bound = 1000

rxn = Reaction('EX_xylt_e')
rxn.name = 'Xylitol exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('xylt_e <=> ')
rxn.lower_bound = 0
rxn.upper_bound = 1000

In [80]:
# NGAM flux reported by David et al. 2008, Analysis of Aspergillus nidulans metabolism at the genome-scale, BMC Genomics, 9, 163-178. 
model.reactions.NGAM.upper_bound = 1.8
model.reactions.NGAM.lower_bound = 1.8
# model.reactions.NGAM.upper_bound = 1000
# model.reactions.NGAM.lower_bound = -1000

In [81]:
for const in model.solver.constraints:
	if const.lb not in (None, 0) or const.ub not in (None, 0):
		print(const.name)
		
for var in model.solver.variables:
	if var.lb > 0:
		print(var.name)

NGAM


In [82]:
# Biomass Exchange Reaction
rxn = Reaction('EX_BIOMASS_c')
rxn.name = 'BIOMASS exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('BIOMASS_c <=> ')
rxn.lower_bound = 0
rxn.upper_bound = 1000

In [83]:
## Add Exchange Reactions for Secretion Products
# Carbonic Acids
rxn = Reaction('EX_mal__L_e')
rxn.name = 'L-Malate exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('mal__L_e -> ')

rxn = Reaction('EX_succ_e')
rxn.name = 'Succinate exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('succ_e -> ')

rxn = Reaction('EX_fum_e')
rxn.name = 'Fumarate exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('fum_e -> ')

rxn = Reaction('EX_itacon_e')
rxn.name = 'Itaconate exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('itacon_e -> ')

In [84]:
# Polyols
rxn = Reaction('EX_mnl_e')
rxn.name = 'Mannitol exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('mnl_e -> ')

In [85]:
model.reactions.EX_arg__L_e.id = 'ARGtex'

In [86]:
# Amino Acids
rxn = Reaction('EX_ala__L_e')
rxn.name = 'Alanine exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('ala__L_e -> ')

rxn = Reaction('EX_arg__L_e')
rxn.name = 'Arginine exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('arg__L_e -> ')

rxn = Reaction('EX_asn__L_e')
rxn.name = 'Asparagine exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('asn__L_e -> ')

rxn = Reaction('EX_asp__L_e')
rxn.name = 'Aspartate exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('asp__L_e -> ')

rxn = Reaction('EX_cys__L_e')
rxn.name = 'Cysteine exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('cys__L_e -> ')

rxn = Reaction('EX_gln__L_e')
rxn.name = 'Glutamine exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('gln__L_e -> ')

rxn = Reaction('EX_gly_e')
rxn.name = 'Glycine exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('gly_e -> ')

rxn = Reaction('EX_his__L_e')
rxn.name = 'Histidine exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('his__L_e -> ')

rxn = Reaction('EX_ile__L_e')
rxn.name = 'Isoleucine exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('ile__L_e -> ')

rxn = Reaction('EX_leu__L_e')
rxn.name = 'Leucine exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('leu__L_e -> ')

rxn = Reaction('EX_lys__L_e')
rxn.name = 'Lysine exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('lys__L_e -> ')

rxn = Reaction('EX_met__L_e')
rxn.name = 'Methionine exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('met__L_e -> ')

rxn = Reaction('EX_phe__L_e')
rxn.name = 'Phenylalanine exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('phe__L_e -> ')

rxn = Reaction('EX_pro__L_e')
rxn.name = 'Proline exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('pro__L_e -> ')

rxn = Reaction('EX_ser__L_e')
rxn.name = 'Serine exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('ser__L_e -> ')

rxn = Reaction('EX_thr__L_e')
rxn.name = 'Threonine exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('thr__L_e -> ')

rxn = Reaction('EX_trp__L_e')
rxn.name = 'Tryptophane exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('trp__L_e -> ')

rxn = Reaction('EX_tyr__L_e')
rxn.name = 'Tyrosine exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('tyr__L_e -> ')

rxn = Reaction('EX_val__L_e')
rxn.name = 'Valine exchange'
model.add_reaction(rxn)
rxn.build_reaction_from_string('val__L_e -> ')

In [87]:
model.reactions.get_by_id('BIOMASS__45__Formation').add_metabolites({model.metabolites.BIOMASS_c:-1},combine=True)

In [88]:
model.reactions.get_by_id('BIOMASS__45__Formation').id = 'BIOMASS_REACTION'

In [89]:
model.change_objective(model.reactions.BIOMASS_REACTION)

In [90]:
sol = model.solve()

In [91]:
sol.f

0.3552831875120829

In [92]:
# for met in model.metabolites:
#     met_id = met.id
#     descriptive_name = met.name
#     if met.id.endswith('_e'):
#         try: 
#             model.reactions.get_by_id('EX_' + met_id)
#         except KeyError:
#             rxn = Reaction('EX_' + met_id)
#             rxn.name = '{} exchange'.format(descriptive_name)
#             rxn.id = 'EX_' + met_id
#             model.add_reaction(rxn)
#             rxn.reaction = met_id + '-->'
#             print rxn
            

In [93]:
solution = foo.show_uptake_excretion(model,model.reactions.BIOMASS_REACTION)

EX_o2_e -12.8556314123
EX_co2_e 3.49043531261
EX_nh4_e -3.27220938926
EX_pi_e -0.856957259607
EX_so4_e -0.029133221376
EX_h2o_e 25.0
EX_nac_e 0.0562306700875
EX_glc_A_e -8.2524844281
EX_mal__L_e 6.75306626714
EX_ala__L_e 0.72829294588
BIOMASS_REACTION 0.355283187512


In [94]:
    for met in [x.id for x in model.reactions.BIOMASS_REACTION.reactants]:
        model_copy = model.copy()
        model_copy.add_reaction(Reaction('DM_test'))
        model_copy.reactions.DM_test.build_reaction_from_string('%s ->' % (met))
        model_copy.objective = model_copy.reactions.DM_test
        print model_copy.reactions.DM_test.reaction, model_copy.solve().f

LINOLEIC_ACID_c -->  0.0399231717808


met__L_c -->  4.85765765766


his__L_c -->  2.12363250133


PHOSPHATIDYLCHOLINE_c -->  1.91545988258


hdca_c -->  3.04705882353


...

glu__L_c -->  6.475


ile__L_c -->  9.16761904762


MANNOSE__45__6P_c -->  14.0244897959


CPD__45__10330_c -->  20.0


lys__L_c -->  4.70909090909


In [95]:
for rxn in model.reactions:
    if rxn.id.startswith('EX_') and rxn.lower_bound < 0:
        print rxn.id,rxn.lower_bound,rxn.upper_bound

EX_o2_e -1000 1000
EX_co2_e -1000 1000
EX_nh4_e -1000 1000
EX_cl_e -1000 1000
EX_k_e -1000 1000
EX_pi_e -1000 1000
EX_mg2_e -1000 1000
EX_so4_e -1000 1000
EX_h2o_e -1000 1000
EX_zn2_e -1000 1000
EX_fe2_e -1000 1000
EX_cobalt2_e -1000 1000
EX_copper2_e -1000 1000
EX_na1_e -1000 1000
EX_ca2_e -1000 1000
EX_btn_e -5 1000
EX_pnto_R_e -5 1000
EX_nac_e -5 1000
EX_inost_e -5 1000
EX_thm_e -5 1000
EX_pydx_e -5 1000
EX_4abz_e -5 1000
EX_glc_A_e -20 1000


#### Move notes to annotations

In [96]:
for met in model.metabolites:
    try:
        len(met.notes.keys()) > 1
        met_notes = met.notes
        anno_dict = {}
        for key in met_notes.keys():
            if key == 'BIGG':
                anno_dict['bigg.metabolite'] = met_notes[key][0].lstrip('\'').rstrip('\'').split(',')[0]
            elif key == 'KEGG':
                    value_entries = met_notes[key][0].lstrip('\'').rstrip('\'').split(',')
                    if len(value_entries) > 1:
                        anno_dict['kegg.compound'] = [x.lstrip(' ') for x in value_entries]
                    else:
                        anno_dict['kegg.compound'] = value_entries[0].lstrip(' ')
            elif key == 'SEED':
                value_entries = met_notes[key][0].lstrip('\'').rstrip('\'').split(',')
                if len(value_entries) > 1:
                    anno_dict['seed.compound'] = [x for x in value_entries]
                else:
                    anno_dict['seed.compound'] = value_entries[0]
            elif key == 'MXNREF':
                value_entries = met_notes[key][0].lstrip('\'').rstrip('\'').split(',')
                if len(value_entries) > 1:
                    anno_dict['metanetx.chemical'] = [key+':'+x.lstrip(' ') for x in value_entries]
                else:
                    anno_dict['metanetx.chemical'] = value_entries[0]
            elif key == 'BIOCYC':
                value_entries = met_notes[key][0].lstrip('\'').rstrip('\'').split(',')
                if len(value_entries) > 1:
                    anno_dict['biocyc'] = ['META:'+x.lstrip(' ') for x in value_entries]
                else:
                    anno_dict['biocyc'] = 'META:'+ value_entries[0].lstrip(' ')
            elif key == 'CHEBI':
                value_entries = met_notes[key][0].lstrip('\'').rstrip('\'').split(',')
                if len(value_entries) > 1:
                    anno_dict['chebi'] = [x.lstrip(' ') for x in value_entries]
                else:
                    anno_dict['chebi'] = value_entries[0].lstrip(' ')
            elif key == 'CAS':
                value_entries = met_notes[key][0].lstrip('\'').rstrip('\'').split(',')
                if len(value_entries) > 1:
                    anno_dict['cas'] = [x.lstrip(' ') for x in value_entries]
                else:
                    anno_dict['cas'] = value_entries[0].lstrip(' ')
            elif key == 'CHEMSPIDER':
                value_entries = met_notes[key][0].lstrip('\'').rstrip('\'').split(',')
                if len(value_entries) > 1:
                    anno_dict['chemspider'] = [x.lstrip(' ') for x in value_entries]
                else:
                    anno_dict['chemspider'] = value_entries[0].lstrip(' ')
            elif key == 'LIPID-MAPS':
                value_entries = met_notes[key][0].lstrip('\'').rstrip('\'').split(',')
                if len(value_entries) > 1:
                    anno_dict['lipidmaps'] = [x.lstrip(' ') for x in value_entries]
                else:
                    anno_dict['lipidmaps'] = value_entries[0].lstrip(' ')
            elif key == 'PUBCHEM':
                value_entries = met_notes[key][0].lstrip('\'').rstrip('\'').split(',')
                if len(value_entries) > 1:
                    anno_dict['pubchem.compound'] = [x.lstrip(' ') for x in value_entries]
                else:
                    anno_dict['pubchem.compound'] = value_entries[0].lstrip(' ')
            elif key == 'SMILES':
                pass
            elif key == 'BRENDA':
                pass
            elif key == 'INCHI':
                value_entries = met_notes[key][0].lstrip('\'').rstrip('\'')
                anno_dict['inchi'] = met_notes[key][0].lstrip('\'').rstrip('\'')
            elif key == 'HMDB':
                value_entries = met_notes[key][0].lstrip('\'').rstrip('\'').split(',')
                if len(value_entries) > 1:
                    anno_dict[key.lower()] = [x.lstrip(' ') for x in value_entries]
                else:
                    anno_dict[key.lower()] = value_entries[0].lstrip(' ')
            else:
                value_entries = met_notes[key][0].lstrip('\'').rstrip('\'').split(',')
                if len(value_entries) > 1:
                    anno_dict[key.lower()] = [key+':'+x.lstrip(' ') for x in value_entries]
                else:
                    anno_dict[key.lower()] = key+':'+value_entries[0].lstrip(' ')
        met.annotation = anno_dict
    except:
        pass

In [97]:
for met in model.metabolites:
    try:
        for key in met.notes.keys():
            if key not in ['SMILES']:
                met.notes.pop(key)
    except:
        print '{1} has no notes section'.format(met)

In [98]:
for rxn in model.reactions:
    try:
        len(rxn.notes.keys()) > 1
        rxn_notes = rxn.notes
        anno_dict = {}
        for key in rxn_notes.keys():
            if key == 'BIGG':
                anno_dict['bigg.reaction'] = rxn_notes[key][0].lstrip('\'').rstrip('\'').split(',')[0]
            elif key == 'KEGG':
                    value_entries = rxn_notes[key][0].lstrip('\'').rstrip('\'').split(',')
                    if len(value_entries) > 1:
                        anno_dict['kegg.reaction'] = [x.lstrip(' ') for x in value_entries]
                    else:
                        anno_dict['kegg.reaction'] = value_entries[0].lstrip(' ')
            elif key == 'SEED':
                value_entries = rxn_notes[key][0].lstrip('\'').rstrip('\'').split(',')
                if len(value_entries) > 1:
                    anno_dict['seed'] = [x.lstrip(' ') for x in value_entries]
                else:
                    anno_dict['seed'] = value_entries[0].lstrip(' ')
            elif key == 'MXNREF':
                value_entries = rxn_notes[key][0].lstrip('\'').rstrip('\'').split(',')
                if len(value_entries) > 1:
                    anno_dict['metanetx.reaction'] = [key+':'+x.lstrip(' ') for x in value_entries]
                else:
                    anno_dict['metanetx.reaction'] = value_entries[0]
            elif key == 'BIOCYC':
                value_entries = rxn_notes[key][0].lstrip('\'').rstrip('\'').split(',')
                if len(value_entries) > 1:
                    anno_dict['biocyc'] = ['META:'+x.lstrip(' ') for x in value_entries]
                else:
                    anno_dict['biocyc'] = 'META:'+ value_entries[0].lstrip(' ')
            elif key == 'GENE ASSOCIATION':
                pass
            elif key == 'CONFIDENCE LEVEL':
                pass
            elif key == 'NOTES':
                pass
            elif key == 'INHIBITOR':
                pass
            elif key == 'LOCALIZATION':
                pass
            elif key == 'COFACTOR':
                pass
            elif key == 'SUBSYSTEM':
                pass
            elif key == 'BRENDA':
                pass
            elif key == 'EC NUMBER':
                value_entries = rxn_notes[key][0].lstrip('\'').rstrip('\'').split(';')
                if len(value_entries) > 1:
                    anno_dict['brenda'] = [x.lstrip(' ') for x in value_entries]
                else:
                    anno_dict['brenda'] = value_entries[0].lstrip(' ')
            elif key == 'REACTOME':
                value_entries = rxn_notes[key][0].lstrip('\'').rstrip('\'').split(',')
                if len(value_entries) > 1:
                    anno_dict['reactome'] = [x.lstrip(' ') for x in value_entries]
                else:
                    anno_dict['reactome'] = value_entries[0].lstrip(' ')
            elif key == 'RHEA':
                value_entries = rxn_notes[key][0].lstrip('\'').rstrip('\'').split(',')
                if len(value_entries) > 1:
                    anno_dict['rhea'] = [x.lstrip(' ') for x in value_entries]
                else:
                    anno_dict['rhea'] = value_entries[0].lstrip(' ')
            else:
                value_entries = rxn_notes[key][0].lstrip('\'').rstrip('\'').split(',')
                if len(value_entries) > 1:
                    anno_dict[key.lower()] = [key+':'+x.lstrip(' ') for x in value_entries]
                else:
                    anno_dict[key.lower()] = key+':'+value_entries[0].lstrip(' ')
        rxn.annotation = anno_dict
    except:
        pass

In [99]:
for rxn in model.reactions:
    if type(rxn.notes) == dict:
        for key in rxn.notes.keys():
            if key in ['CONFIDENCE LEVEL', 'LOCALIZATION', 'SUBSYSTEM','INHIBITOR','COFACTOR','NOTES']:
                value_entries = rxn.notes[key][0].strip('\'').split(',')
                if len(value_entries) > 1:
                    rxn.notes[key] = [x.lstrip(' ') for x in value_entries]
                else:
                    rxn.notes[key] = value_entries[0]
            else:
                rxn.notes.pop(key)

In [100]:
from cobra.manipulation.annotate import add_SBO

In [101]:
add_SBO(model)

### Export the model as JSON and XML using the most recent formats

In [103]:
import cobra as cobra

In [106]:
cobra.io.write_sbml_model(model, 'iCL_Conversion1.xml', use_fbc_package=True)
cobra.io.save_json_model(model, 'iCL_Conversion1.json',pretty = True)

In [ ]:
emap = escher.Builder(model=model)
emap.display_in_browser()